In [1]:
import pandas as pd
import utils.fetcher_utils as fetcher
import utils.pipeline_util as pu
import utils.transformer_util as tu

In [2]:
# set pandas options
pd.set_option('display.max_columns', None)

In [3]:
# fetch the data 
df = fetcher.aquireIMDbDataFrame()
df.head()

Reading data from ../resources/movie_metadata.csv


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [4]:
# preprocess the data
# processed_df = preproc.preprocess_data(df)
# processed_df.head()
trained_df, X_test, model = pu.run_pipeline(df, True)


Mean Squared Error: 0.539809330472059
R-squared: 0.4910815174953892


In [5]:
X_test

,Action,Adventure,Comedy,Crime,Drama,Family,Fantasy,Horror,Romance,Sci-Fi,Thriller,actor_total_facebook_likes,budget,cast_total_facebook_likes,director_facebook_likes,director_frequency,duration,facenumber_in_poster,gross,movie_facebook_likes,num_critic_for_reviews,num_user_for_reviews,num_voted_users,other_genre,rating_bin,title_year,total_actor_frequency
2961,0,0,1,1,0,1,0,0,0,0,0,2070.0,12000000.0,4551,25.0,5,91.0,0.0,17292381.0,371,37.0,69.0,5794,0,0,2001.0,22
143,1,1,1,0,0,1,0,0,0,1,0,1916.0,150000000.0,2652,25.0,2,88.0,0.0,21379315.0,0,132.0,112.0,17590,1,0,2011.0,14
1480,0,0,1,0,0,1,0,0,0,0,0,4734.0,35000000.0,6327,16.0,3,92.0,1.0,17596256.0,0,101.0,84.0,12399,0,0,2010.0,22
2271,0,0,1,0,0,0,0,0,1,0,0,12687.0,20000000.0,15713,20.0,1,117.0,1.0,13987482.0,0,151.0,91.0,58957,0,2,2011.0,29
1853,1,1,1,0,0,1,0,0,1,1,0,2522.0,26000000.0,3742,8.0,2,100.0,4.0,23222861.0,167,55.0,44.0,11092,0,0,2004.0,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4251,0,0,0,0,0,0,0,1,0,0,0,662.0,2000000.0,1062,421.0,1,100.0,0.0,14673301.0,45000,533.0,631.0,112899,1,2,2014.0,3
4737,0,0,0,0,1,0,0,0,0,0,0,372.0,500000.0,407,7.0,2,86.0,0.0,1114943.0,0,46.0,74.0,3562,1,2,2000.0,4
23,0,1,0,0,0,0,1,0,0,0,0,6745.0,225000000.0,9152,0.0,12,186.0,6.0,258355354.0,83000,509.0,951.0,483540,0,1,2013.0,15
745,0,0,0,0,0,0,0,0,0,1,1,12697.0,48000000.0,13654,0.0,9,91.0,0.0,64505912.0,6000,323.0,1264.0,163130,0,2,2008.0,29


In [6]:
trained_df

,Action,Adventure,Comedy,Crime,Drama,Family,Fantasy,Horror,Romance,Sci-Fi,Thriller,actor_total_facebook_likes,budget,cast_total_facebook_likes,director_facebook_likes,director_frequency,duration,facenumber_in_poster,gross,movie_facebook_likes,num_critic_for_reviews,num_user_for_reviews,num_voted_users,other_genre,rating_bin,title_year,total_actor_frequency
1909,1,0,0,1,1,0,0,0,0,0,1,25469.0,25000000.0,26625,175.0,3,109.0,3.0,66489425.0,0,270.0,161.0,101977,0,2,2012.0,37
1255,0,0,1,0,1,0,0,0,0,0,0,841.0,40000000.0,1070,98.0,5,95.0,1.0,37101011.0,0,147.0,103.0,30394,0,1,2012.0,8
1002,1,1,0,0,0,0,0,0,1,1,1,960.0,40000000.0,1300,487.0,5,125.0,3.0,26616999.0,43000,298.0,347.0,92461,0,1,2013.0,6
1864,1,0,0,0,0,0,0,0,0,1,1,17066.0,26000000.0,17847,763.0,4,117.0,0.0,12276810.0,11000,178.0,126.0,57674,1,1,2015.0,30
1615,1,0,0,1,1,0,0,0,0,1,1,36502.0,30000000.0,37387,0.0,2,119.0,0.0,66468315.0,75000,589.0,667.0,428916,0,2,2012.0,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2312,0,0,0,0,1,0,0,0,0,0,0,19956.0,20000000.0,21200,150.0,6,116.0,2.0,6002756.0,13000,116.0,75.0,24997,1,0,2012.0,32
1431,0,0,1,0,0,1,0,0,0,0,0,50931.0,35000000.0,53370,12.0,4,88.0,2.0,49474048.0,0,124.0,120.0,31191,0,0,2009.0,44
1486,0,0,0,0,1,0,0,0,0,0,0,19331.0,35000000.0,19805,529.0,9,111.0,3.0,11405825.0,0,119.0,208.0,28542,0,2,2001.0,40
1737,0,0,1,0,1,0,0,0,1,0,0,2578.0,30000000.0,5404,0.0,1,128.0,3.0,37036404.0,0,148.0,259.0,44816,0,1,2000.0,10
